In [1]:
import joblib
import random
import pandas as pd

# import nltk
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('stopwords')
# nltk.download('averaged_perceptron_tagger')

from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer

import warnings
warnings.filterwarnings('ignore')

from helpers.preprocessing import PreProcessing

## Proprocessing and Save Vectorizer

In [ ]:
data = pd.read_excel("data/symp.xlsx")
symptoms = data['q_11'].to_numpy()

processing = PreProcessing()
symptoms_preprocess = processing.transform(symptoms)
symptoms_preprocess

In [33]:
vectorizer = CountVectorizer(
    max_df=len(symptoms_preprocess), 
    min_df=0,
    decode_error='ignore',
    binary=True
)

data_vectorizer = pd.DataFrame(
    vectorizer.fit_transform(symptoms_preprocess).toarray(), 
    columns=vectorizer.get_feature_names_out()
)
# buat random, karena datanya tidak ada
disorder = ['Bipolar', 'Non Bipolar']
data_vectorizer['disorder'] =  random.choices(disorder, k=data_vectorizer.shape[0])


data_vectorizer.to_csv('data/data_vectorizer.csv', index=False)
joblib.dump(vectorizer, 'modeling/vectorizer.save') 

['modeling/vectorizer.save']

## Make Simple Model

In [58]:
data = pd.read_csv('data/data_vectorizer.csv')

target_column = 'disorder'
X = data.drop(target_column, axis=1).to_numpy()

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data[target_column])


model = DecisionTreeClassifier(random_state = 50, criterion = "gini")
model.fit(X, y)

DecisionTreeClassifier(random_state=50)

In [57]:
# try to predict
prediction = model.predict(X[0:2])
label_encoder.inverse_transform(prediction)

array(['Bipolar', 'Non Bipolar'], dtype=object)

In [59]:
# save model dan encoder
joblib.dump(label_encoder, "modeling/label_encoder.save")
joblib.dump(model, 'modeling/decision_tree_model.pkl')

['modeling/decision_tree_model.pkl']

## Try to Predict New Data

In [2]:
symptoms = "I feel very anxious, sometimes I want to suicide"

vectorizer = joblib.load('modeling/vectorizer.save')
label_encoder = joblib.load('modeling/label_encoder.save')
model = joblib.load('modeling/decision_tree_model.pkl')

preprocessing = PreProcessing()
symptoms_preprocess = preprocessing.transform(symptoms)

X = vectorizer.transform([symptoms_preprocess]).toarray()
prediction = model.predict(X)
prediction = label_encoder.inverse_transform(prediction)[0]
prediction

b'Skipping line 8: expected 46 fields, saw 56\n'
b'Skipping line 8: expected 25 fields, saw 29\n'
Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ASUS\AppData\Local\Temp\jieba.cache
Loading model cost 0.839 seconds.
Prefix dict has been built successfully.


'Bipolar'